In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By

from tqdm import tqdm

import time

In [10]:
driver = webdriver.Chrome()

In [11]:
all_quotes = []

In [12]:
for i in tqdm(range(1, 101)):
    url = f"https://www.goodreads.com/author/quotes/656983.J_R_R_Tolkien?page={i}"

    driver.get(url)

    time.sleep(0.1)

    quotes_per_page = driver.find_elements(By.CLASS_NAME, "quoteText")
    all_quotes.extend([quote.text for quote in quotes_per_page])
    # print(quotes_per_page[-1].text)
    quotes_per_page.clear()

driver.quit()

100%|██████████| 100/100 [04:19<00:00,  2.59s/it]


In [13]:
print(len(all_quotes))

3000


In [14]:
print(all_quotes[0])

“All that is gold does not glitter,
Not all those who wander are lost;
The old that is strong does not wither,
Deep roots are not reached by the frost.

From the ashes a fire shall be woken,
A light from the shadows shall spring;
Renewed shall be blade that was broken,
The crownless again shall be king.”
― J.R.R. Tolkien, The Fellowship of the Ring


In [15]:
for i in all_quotes[30:65]:
    print(i)

“Far over the misty mountains cold
To dungeons deep and caverns old
We must away ere break of day
To seek the pale enchanted gold.

The dwarves of yore made mighty spells,
While hammers fell like ringing bells
In places deep, where dark things sleep,
In hollow halls beneath the fells.

For ancient king and elvish lord
There many a gleaming golden hoard
They shaped and wrought, and light they caught
To hide in gems on hilt of sword.

On silver necklaces they strung
The flowering stars, on crowns they hung
The dragon-fire, in twisted wire
They meshed the light of moon and sun.

Far over the misty mountains cold
To dungeons deep and caverns old
We must away, ere break of day,
To claim our long-forgotten gold.

Goblets they carved there for themselves
And harps of gold; where no man delves
There lay they long, and many a song
Was sung unheard by men or elves.

The pines were roaring on the height,
The wind was moaning in the night.
The fire was red, it flaming spread;
The trees like torche

In [16]:
print(all_quotes[-1])

“For Morgoth had long prepared his force in secret, while ever the malice of his heart grew greater, and his hatred of the Noldor more bitter; and he desired not only to end his foes but to destroy also and defile the lands that they had taken and made fair.”
― J.R.R. Tolkien, The Silmarillion


#### Manually extracting data from page 2

In [17]:
url = f"https://www.goodreads.com/author/quotes/656983.J_R_R_Tolkien?page=2"

driver = webdriver.Chrome()

driver.get(url)

quotes_per_page = driver.find_elements(By.CLASS_NAME, "quoteText")
all_quotes.extend([quote.text for quote in quotes_per_page])

driver.quit()

In [24]:
len(all_quotes)

3030

In [23]:
for i in all_quotes[3000:]:
    print(i)
    print("*"*50)

“Far over the misty mountains cold
To dungeons deep and caverns old
We must away ere break of day
To seek the pale enchanted gold.

The dwarves of yore made mighty spells,
While hammers fell like ringing bells
In places deep, where dark things sleep,
In hollow halls beneath the fells.

For ancient king and elvish lord
There many a gleaming golden hoard
They shaped and wrought, and light they caught
To hide in gems on hilt of sword.

On silver necklaces they strung
The flowering stars, on crowns they hung
The dragon-fire, in twisted wire
They meshed the light of moon and sun.

Far over the misty mountains cold
To dungeons deep and caverns old
We must away, ere break of day,
To claim our long-forgotten gold.

Goblets they carved there for themselves
And harps of gold; where no man delves
There lay they long, and many a song
Was sung unheard by men or elves.

The pines were roaring on the height,
The wind was moaning in the night.
The fire was red, it flaming spread;
The trees like torche

In [27]:
import pandas as pd

raw_quotes_df = pd.DataFrame(all_quotes, columns=['quote'])
raw_quotes_df

,quote
0,"“All that is gold does not glitter,\nNot all t..."
1,“Not all those who wander are lost.”\n― J.R.R....
2,"“I wish it need not have happened in my time,""..."
3,“I don't know half of you half as well as I sh...
4,“All we have to decide is what to do with the ...
...,...
3025,"“Home is behind, the world ahead,\nAnd there a..."
3026,"“It is not despair, for despair is only for th..."
3027,"“There is more in you of good than you know, c..."
3028,"“Good Morning!"" said Bilbo, and he meant it. T..."


In [49]:
raw_quotes_df["quote"][0]

'“All that is gold does not glitter,\nNot all those who wander are lost;\nThe old that is strong does not wither,\nDeep roots are not reached by the frost.\n\nFrom the ashes a fire shall be woken,\nA light from the shadows shall spring;\nRenewed shall be blade that was broken,\nThe crownless again shall be king.”\n― J.R.R. Tolkien, The Fellowship of the Ring'

In [55]:
quotes_df = raw_quotes_df["quote"].str.rpartition("―")[0]

In [56]:
quotes_df

0       “All that is gold does not glitter,\nNot all t...
1                  “Not all those who wander are lost.”\n
2       “I wish it need not have happened in my time,"...
3       “I don't know half of you half as well as I sh...
4       “All we have to decide is what to do with the ...
                              ...                        
3025    “Home is behind, the world ahead,\nAnd there a...
3026    “It is not despair, for despair is only for th...
3027    “There is more in you of good than you know, c...
3028    “Good Morning!" said Bilbo, and he meant it. T...
3029    “In sorrow we must go, but not in despair. Beh...
Name: 0, Length: 3030, dtype: object

In [98]:
type(quotes_df)

pandas.core.series.Series

In [99]:
# remove empty quotes
quotes_df.drop(quotes_df.index[53:60], axis=0, inplace=True)

In [100]:
quotes_df = quotes_df.reset_index(drop=True).squeeze()
quotes_df

0       All that is gold does not glitter,\nNot all th...
1                    Not all those who wander are lost.\n
2       I wish it need not have happened in my time," ...
3       I don't know half of you half as well as I sho...
4       All we have to decide is what to do with the t...
                              ...                        
3004    “Home is behind, the world ahead,\nAnd there a...
3005    “It is not despair, for despair is only for th...
3006    “There is more in you of good than you know, c...
3007    “Good Morning!" said Bilbo, and he meant it. T...
3008    “In sorrow we must go, but not in despair. Beh...
Name: 0, Length: 3009, dtype: object

In [129]:
quotes_df = quotes_df.str.replace('[“”]', '', regex=True)

In [130]:
print(quotes_df[2458])

If I choose to send thee, Tuor son of Huor, then believe not that thy one sword is not worth the sending. For the valour of the Edain the Elves shall ever remember as the ages lengthen, marvelling that they gave life so freely of which they had on earth so little. But it is not for thy valour only that I send thee, but to bring into the world a hope beyond thy sight, and a light that shall pierce the darkness.



In [131]:
quotes_df.to_csv("quotes_df.txt", index=False, header=False, encoding="utf-8")